# Análise Comparativa de Modelos de Machine Learning
## Classificação de Depressão em Estudantes

### Objetivo do Projeto
Este projeto implementa e compara três diferentes algoritmos de machine learning para classificação binária de depressão em estudantes:
- **Support Vector Machine (SVM)**
- **Árvore de Decisão**
- **Redes Neurais**

### Metodologia
- Dataset balanceado (50% / 50%)
- Divisão treino/teste: 80% / 20%
- Validação cruzada para otimização de hiperparâmetros
- Métricas: Accuracy, Precision, Recall, F1-Score

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

In [38]:
# Função para carregar arquivo de métricas
def load_metrics(file_path):
    try:
        with open(file_path, 'rb') as f:
            return pickle.load(f)
    except Exception as e:
        print(f"Erro ao carregar {file_path}: {e}")
        return None

## 1. Resultados dos Modelos

### Consolidação dos Resultados Obtidos
Os resultados abaixo foram coletados dos notebooks individuais de cada modelo:

In [50]:
# Caminhos para os arquivos de métricas
svm_metrics_path = "modelos/models/metricas_svm.pkl"
arvore_metrics_path = "modelos/models/arvore_decisao_info.pkl"
nn_metrics_path = "modelos/models/metricas_modelo_rede_neurais.pkl"

# Carregar métricas dos modelos
svm_metrics = load_metrics(svm_metrics_path)
arvore_metrics = load_metrics(arvore_metrics_path)
nn_metrics = load_metrics(nn_metrics_path)

In [51]:
# Converter métricas para o formato do DataFrame
svm_results = {
    'Modelo': 'SVM',
    'Accuracy': svm_metrics['accuracy'],
    'Precision_0': svm_metrics['precision_0'],
    'Recall_0': svm_metrics['recall_0'],
    'F1_Score_0': svm_metrics['f1_score_0'],
    'Precision_1': svm_metrics['precision_1'],
    'Recall_1': svm_metrics['recall_1'],
    'F1_Score_1': svm_metrics['f1_score_1'],
    'Ein': svm_metrics['ein'],
    'Eout': svm_metrics['eout']
}

# Adaptação para o formato de métricas da Árvore de Decisão
arvore_results = {
    'Modelo': 'Árvore de Decisão',
    'Accuracy': arvore_metrics['accuracy'],
    'Precision_0': arvore_metrics['precision_0'] if 'precision_0' in arvore_metrics else arvore_metrics['precision'],
    'Recall_0': arvore_metrics['recall_0'] if 'recall_0' in arvore_metrics else arvore_metrics['recall'],
    'F1_Score_0': arvore_metrics['f1_score_0'] if 'f1_score_0' in arvore_metrics else arvore_metrics['f1_score'],
    'Precision_1': arvore_metrics['precision_1'] if 'precision_1' in arvore_metrics else arvore_metrics['precision'],
    'Recall_1': arvore_metrics['recall_1'] if 'recall_1' in arvore_metrics else arvore_metrics['recall'],
    'F1_Score_1': arvore_metrics['f1_score_1'] if 'f1_score_1' in arvore_metrics else arvore_metrics['f1_score'],
    'Ein': arvore_metrics['ein'],
    'Eout': arvore_metrics['eout']
}

redes_results = {
    'Modelo': 'Redes Neurais',
    'Accuracy': nn_metrics['accuracy'], 
    'Precision_0': nn_metrics['precision_0'],
    'Recall_0': nn_metrics['recall_0'],
    'F1_Score_0': nn_metrics['f1_score_0'],
    'Precision_1': nn_metrics['precision_1'],
    'Recall_1': nn_metrics['recall_1'],
    'F1_Score_1': nn_metrics['f1_score_1'],
    'Ein': nn_metrics['ein'],
    'Eout': nn_metrics['eout']
}

In [52]:

# Criar DataFrame com os resultados
results_df = pd.DataFrame([svm_results, arvore_results, redes_results])

print("=== RESULTADOS CONSOLIDADOS ===")
print(results_df[['Modelo', 'Accuracy', 'F1_Score_0', 'F1_Score_1', 'Recall_0', 'Recall_1']])

=== RESULTADOS CONSOLIDADOS ===
              Modelo  Accuracy  F1_Score_0  F1_Score_1  Recall_0  Recall_1
0                SVM  0.825142    0.822740    0.827479  0.807018  0.843473
1  Árvore de Decisão  0.817580    0.817600    0.817600  0.817580  0.817580
2      Redes Neurais  0.824827    0.826223    0.826223  0.829868  0.829868


In [53]:
# Análise detalhada dos modelos
print("=== ANÁLISE COMPARATIVA DETALHADA ===\n")

# 1. Comparação das métricas principais
print("1. MÉTRICAS PRINCIPAIS:")
print("-" * 50)
for index, row in results_df.iterrows():
    print(f"{row['Modelo']:<20} | Accuracy: {row['Accuracy']:.3f} | F1-Score Médio: {(row['F1_Score_0'] + row['F1_Score_1'])/2:.3f}")

# 2. Análise de overfitting (Ein vs Eout)
print("\n2. ANÁLISE DE OVERFITTING (Ein vs Eout):")
print("-" * 50)
results_df['Overfitting'] = results_df['Eout'] - results_df['Ein']
for index, row in results_df.iterrows():
    status = "✓ Bom" if abs(row['Overfitting']) < 0.02 else "⚠ Overfitting" if row['Overfitting'] > 0.02 else "⚠ Underfitting"
    print(f"{row['Modelo']:<20} | Ein: {row['Ein']:.4f} | Eout: {row['Eout']:.4f} | Diff: {row['Overfitting']:.4f} | {status}")

# 3. Balanceamento entre classes
print("\n3. BALANCEAMENTO ENTRE CLASSES:")
print("-" * 50)
for index, row in results_df.iterrows():
    f1_diff = abs(row['F1_Score_0'] - row['F1_Score_1'])
    balance_status = "✓ Balanceado" if f1_diff < 0.02 else "⚠ Desbalanceado"
    print(f"{row['Modelo']:<20} | F1 Classe 0: {row['F1_Score_0']:.3f} | F1 Classe 1: {row['F1_Score_1']:.3f} | Diff: {f1_diff:.3f} | {balance_status}")

# 4. Cálculo do score final ponderado
print("\n4. SCORE FINAL PONDERADO:")
print("-" * 50)

def calculate_final_score(row):
    # Peso para accuracy (30%)
    accuracy_score = row['Accuracy'] * 0.3
    
    # Peso para F1-Score médio (40%)
    f1_mean = (row['F1_Score_0'] + row['F1_Score_1']) / 2
    f1_score = f1_mean * 0.4
    
    # Penalidade por overfitting (20%)
    overfitting_penalty = max(0, 0.2 - abs(row['Overfitting']) * 10) * 0.2
    
    # Penalidade por desbalanceamento (10%)
    balance_penalty = max(0, 0.1 - abs(row['F1_Score_0'] - row['F1_Score_1']) * 5) * 0.1
    
    return accuracy_score + f1_score + overfitting_penalty + balance_penalty

results_df['Score_Final'] = results_df.apply(calculate_final_score, axis=1)

# Ordenar por score final
results_df_sorted = results_df.sort_values('Score_Final', ascending=False)

for index, row in results_df_sorted.iterrows():
    print(f"{row['Modelo']:<20} | Score Final: {row['Score_Final']:.4f}")

# 5. Determinação do melhor modelo
print(f"\n{'='*60}")
print("RESULTADO FINAL:")
print(f"{'='*60}")

best_model = results_df_sorted.iloc[0]
print(f"MELHOR MODELO: {best_model['Modelo']}")
print(f"Score Final: {best_model['Score_Final']:.4f}")
print(f"Accuracy: {best_model['Accuracy']:.3f}")
print(f"F1-Score Médio: {(best_model['F1_Score_0'] + best_model['F1_Score_1'])/2:.3f}")
print(f"Overfitting: {best_model['Overfitting']:.4f} {'(Controlado)' if abs(best_model['Overfitting']) < 0.02 else '(Presente)'}")

# 6. Justificativa da escolha
print("\nJUSTIFICATIVA:")
print("-" * 60)

if best_model['Modelo'] == 'SVM':
    print("✓ Excelente accuracy e F1-score")
    print("✓ Boa generalização (Ein vs Eout)")
    print("✓ Balanceamento adequado entre classes")
elif best_model['Modelo'] == 'Redes Neurais':
    print("✓ Melhor generalização (menor Eout)")
    print("✓ Boa performance geral")
    print("✓ Flexibilidade para dados complexos")
else:
    print("✓ Interpretabilidade do modelo")
    print("✓ Simplicidade e eficiência")
    print("✓ Boa performance geral")

print(f"\nCONCLUSÃO: O modelo {best_model['Modelo']} é o mais adequado para este problema de classificação de depressão.")

=== ANÁLISE COMPARATIVA DETALHADA ===

1. MÉTRICAS PRINCIPAIS:
--------------------------------------------------
SVM                  | Accuracy: 0.825 | F1-Score Médio: 0.825
Árvore de Decisão    | Accuracy: 0.818 | F1-Score Médio: 0.818
Redes Neurais        | Accuracy: 0.825 | F1-Score Médio: 0.826

2. ANÁLISE DE OVERFITTING (Ein vs Eout):
--------------------------------------------------
SVM                  | Ein: 0.1689 | Eout: 0.1749 | Diff: 0.0060 | ✓ Bom
Árvore de Decisão    | Ein: 0.1790 | Eout: 0.1824 | Diff: 0.0034 | ✓ Bom
Redes Neurais        | Ein: 0.1760 | Eout: 0.1752 | Diff: -0.0008 | ✓ Bom

3. BALANCEAMENTO ENTRE CLASSES:
--------------------------------------------------
SVM                  | F1 Classe 0: 0.823 | F1 Classe 1: 0.827 | Diff: 0.005 | ✓ Balanceado
Árvore de Decisão    | F1 Classe 0: 0.818 | F1 Classe 1: 0.818 | Diff: 0.000 | ✓ Balanceado
Redes Neurais        | F1 Classe 0: 0.826 | F1 Classe 1: 0.826 | Diff: 0.000 | ✓ Balanceado

4. SCORE FINAL PONDERAD